In [1]:
import os

# Path to your dataset
dataset_path = r"C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath\vehicle_dataset\q2_Images"

# List files inside the folder
print("Files in dataset:", os.listdir(dataset_path))


Files in dataset: ['20250901.104645.912.J001A1.SL.FR.NaN.NaN.jpg', '20250901.153410.209.J001A1.SL.FR.NaN.NaN.jpg', '20250904.072631.739.J001A1.SL.FR.NaN.NaN.jpg', '20250910.200345.132.J001A1.FL.FR.NaN.NaN.jpg', '20250915.133513.973.J001A1.FL.FR.PUTRAJAYA9558.NaN.jpg', '20250915.154453.318.J001A1.FL.RE.L2233.NaN.jpg', '20250915.164521.488.J001A1.SL.FR.MALAYSIA1834.NaN.jpg', '20250915.172302.451.J001A1.FL.RE.PATRIOT1513.NaN.jpg', '20250917.071629.915.J001A1.SL.FR.JUM5353.NaN.jpg', '20250917.071700.908.J001A1.SL.FR.WB5138N.NaN.jpg', '20250917.071924.072.J001A1.SL.RE.JXV718.NaN.jpg', '20250917.071940.201.J001A1.FL.FR.JXC2052.NaN.jpg', '20250917.072920.154.J001A1.SL.FR.JVM3445.NaN.jpg', '20250917.073006.305.J001A1.SL.FR.JTQ5254.NaN.jpg', '20250917.085912.220.J001A1.SL.RE.JJ1975.NaN.jpg', '20250917.090217.279.J001A1.FL.FR.UT4583.NaN.jpg', '20250917.090217.640.J001A1.FL.RE.JUY4583.NaN.jpg', '20250917.090445.972.J001A1.FL.FR.BPE4281.NaN.jpg', '20250917.090639.150.J001A1.SL.RE.CDD1917.NaN.jpg',

In [7]:
!pip install ultralytics
import ultralytics
ultralytics.checks()


Ultralytics 8.3.203  Python-3.10.18 torch-2.8.0+cpu CPU (Intel Core i5-1035G1 1.00GHz)
Setup complete  (8 CPUs, 7.8 GB RAM, 278.1/288.4 GB disk)


In [2]:
label_path = r"C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath\vehicle_dataset\q2_labels"
sample_file = os.listdir(label_path)[0]

with open(os.path.join(label_path, sample_file), "r") as f:
    print(f.read())


0 0.457473 0.376522 0.395380 0.662609
1 0.456793 0.650435 0.201087 0.069565
2 0.450000 0.576522 0.057065 0.064348


In [5]:
images = [os.path.splitext(f)[0] for f in os.listdir(dataset_path)]
labels = [os.path.splitext(f)[0] for f in os.listdir(label_path)]

print("Unmatched images:", set(images) - set(labels))
print("Unmatched labels:", set(labels) - set(images))


Unmatched images: set()
Unmatched labels: set()


In [3]:
yaml_content = """
train: /content/drive/MyDrive/vehicle_attribute_identification/vehicle_dataset/images/train
val: /content/drive/MyDrive/vehicle_attribute_identification/vehicle_dataset/images/val

nc: 14
names: [ 'truck', 'license_plate', 'logo', 'car', 'bike', 'bus', 'nissan', 'toyota', 'isuzu', 'mitsubishi', 'proton', 'mazda', 'mercedes', 'honda' ]

nc: 14
names: [ 'truck', 'license_plate', 'logo', 'car', 'bike', 'bus', 'nissan', 'toyota', 'isuzu', 'mitsubishi', 'proton', 'mazda', 'mercedes', 'honda' ]
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)
print("created!")



created!


In [4]:
import os, shutil, random

base = r"C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath\vehicle_dataset"

# Make YOLO folders
os.makedirs(f"{base}/images/train", exist_ok=True)
os.makedirs(f"{base}/images/val", exist_ok=True)
os.makedirs(f"{base}/labels/train", exist_ok=True)
os.makedirs(f"{base}/labels/val", exist_ok=True)

# Get all images
images = [f for f in os.listdir(f"{base}/q2_Images") if f.endswith(('.jpg','.png','.jpeg'))]

# Train/val split (80/20)
random.shuffle(images)
split = int(0.8 * len(images))
train_imgs, val_imgs = images[:split], images[split:]

# Move files into correct folders
for img in train_imgs:
    name, ext = os.path.splitext(img)
    shutil.copy(f"{base}/q2_Images/{img}", f"{base}/images/train/{img}")
    shutil.copy(f"{base}/q2_labels/{name}.txt", f"{base}/labels/train/{name}.txt")

for img in val_imgs:
    name, ext = os.path.splitext(img)
    shutil.copy(f"{base}/q2_Images/{img}", f"{base}/images/val/{img}")
    shutil.copy(f"{base}/q2_labels/{name}.txt", f"{base}/labels/val/{name}.txt")

print("✅ Dataset organized into YOLO format")


✅ Dataset organized into YOLO format


In [ ]:
from ultralytics import YOLO

# 1. Load pretrained YOLOv8 model
model = YOLO("yolov8m.pt")   

# 2. Train on CPU
results = model.train(
    data=r"C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath\vehicle_dataset\data.yaml",
    epochs=60,
    batch=8,                 
    imgsz=640,
    device="cuda",            
    workers=4,               
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.0005,
    augment=True,
    mosaic=1.0,
    mixup=0.2,
    project=r"C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath",
    name="yolov8_vehicle_model",
    pretrained=True
)

# 3. Check training results
print("Training finished. Results are saved in:")
print(results.save_dir)

# 4. Evaluate the trained model on validation set
metrics = model.val()
print(metrics)

# 5. Run inference on a test image (just to check quickly)
test_img = r"C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath\vehicle_dataset\q2_Images\20250917.113850.167.J001A1.FL.FR.W8273T.NaN.jpg"
predictions = model.predict(source=test_img, save=True, conf=0.25)

# Annotated result will be saved inside 'runs/detect/predict' folder

Ultralytics 8.3.203  Python-3.10.18 torch-2.8.0+cpu CPU (Intel Core i5-1035G1 1.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath\vehicle_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_vehicle_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, ov

: 

In [ ]:
# improved_vehicle_inference.py
import os
import json
import math
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime

# -------- CONFIG --------
SAVE_DIR = r"C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath"
CONF_THR = 0.25    # detection confidence threshold
IOU_ASSOC_THRESH = 0.25
MAX_TEXT_FONT = 28
os.makedirs(SAVE_DIR, exist_ok=True)

# -------- Auto-find best.pt --------
def get_latest_best_pt(base_dir="runs/detect"):
    """Find the most recent YOLO run with best.pt inside."""
    if not os.path.exists(base_dir):
        raise FileNotFoundError(f"No YOLO runs found in {base_dir}")

    run_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    if not run_dirs:
        raise FileNotFoundError("No training run directories found.")

    # pick the most recently modified run
    latest_run = max(run_dirs, key=os.path.getmtime)
    weights_path = os.path.join(latest_run, "weights", "best.pt")
    if not os.path.exists(weights_path):
        raise FileNotFoundError(f"best.pt not found in {latest_run}")
    return weights_path

try:
    MODEL_PATH = get_latest_best_pt()
except FileNotFoundError as e:
    print("⚠️ Could not auto-locate best.pt. Please train a model first or set MODEL_PATH manually.")
    raise e

# -------- Test image --------
TEST_IMG = r"C:\Users\fayaf\Downloads\FaceRec_Rahmath\Q2_Rahmath\vehicle_dataset\q2_Images\20250917.113850.167.J001A1.FL.FR.W8273T.NaN.jpg"

# -------- Helpers --------
def iou(boxA, boxB):
    # boxes as [x1,y1,x2,y2]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    denom = float(boxAArea + boxBArea - interArea + 1e-9)
    return interArea / denom if denom > 0 else 0.0

def box_center(box):
    x1,y1,x2,y2 = box
    return ((x1+x2)/2.0, (y1+y2)/2.0)

def mean_hsv_color_name(bgr_crop):
    """Return robust color name using HSV mean and heuristics."""
    if bgr_crop is None or bgr_crop.size == 0:
        return "Unknown"
    hsv = cv2.cvtColor(bgr_crop, cv2.COLOR_BGR2HSV)
    h_mean = float(np.mean(hsv[:,:,0]))  # 0..179
    s_mean = float(np.mean(hsv[:,:,1]))  # 0..255
    v_mean = float(np.mean(hsv[:,:,2]))  # 0..255

    # brightness/sat based checks for white/black/gray
    if v_mean > 220 and s_mean < 30:
        return "White"
    if v_mean < 40:
        return "Black"
    if s_mean < 40 and 40 < v_mean < 220:
        return "Gray"

    # convert hue to degrees 0..360
    hue_deg = (h_mean / 179.0) * 360.0

    if (hue_deg <= 15) or (hue_deg > 345):
        return "Red"
    if 15 < hue_deg <= 45:
        return "Orange"
    if 45 < hue_deg <= 75:
        return "Yellow"
    if 75 < hue_deg <= 150:
        return "Green"
    if 150 < hue_deg <= 210:
        return "Cyan"
    if 210 < hue_deg <= 270:
        return "Blue"
    if 270 < hue_deg <= 315:
        return "Purple"
    if 315 < hue_deg <= 345:
        return "Pink"
    return "Unknown"

def safe_int_box(box):
    return [int(max(0, round(x))) for x in box]

def try_load_font(preferred_size=18):
    # Try common TTFs (Colab/Ubuntu/Windows). Fall back to None.
    candidates = [
        "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",  # linux
        "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf",
        "C:/Windows/Fonts/Arial.ttf",  # windows
        "C:/Windows/Fonts/arial.ttf"
    ]
    for path in candidates:
        try:
            font = ImageFont.truetype(path, preferred_size)
            return font
        except Exception:
            continue
    return None  # fallback to cv2.putText if None

def draw_text_pil(img_bgr, text, xy, font=None, text_color=(255,255,255), bg_color=(0,0,0), padding=6):
    """Draw nicer text using PIL and return BGR image"""
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(img_rgb)
    draw = ImageDraw.Draw(pil)

    if font is None:
        font = ImageFont.load_default()

    # Use textbbox instead of textsize
    bbox = draw.textbbox((0, 0), text, font=font)
    text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]

    x, y = xy
    rect = [x, y, x + text_w + padding, y + text_h + padding//2]
    draw.rectangle(rect, fill=bg_color)
    draw.text((x+padding//2, y), text, fill=text_color, font=font)

    out = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
    return out


# -------- Load model & image --------
print("Loading model:", MODEL_PATH)
model = YOLO(MODEL_PATH)

img = cv2.imread(TEST_IMG)
if img is None:
    raise FileNotFoundError("Could not load TEST_IMG: " + TEST_IMG)
h, w, _ = img.shape
orig_img = img.copy()

# -------- Run inference --------
print("Running inference...")
results = model.predict(source=TEST_IMG, conf=CONF_THR, verbose=False)  # list of Results
# results may contain 1 element (image)
if len(results) == 0:
    detections = []
else:
    r = results[0]
    boxes_xyxy = r.boxes.xyxy.cpu().numpy() if hasattr(r.boxes, 'xyxy') else np.array([])
    confs = r.boxes.conf.cpu().numpy() if hasattr(r.boxes, 'conf') else np.array([])
    classes = r.boxes.cls.cpu().numpy().astype(int) if hasattr(r.boxes, 'cls') else np.array([])
    detections = []
    for box, cls, conf in zip(boxes_xyxy, classes, confs):
        detections.append({
            "box": [float(box[0]), float(box[1]), float(box[2]), float(box[3])],
            "class": int(cls),
            "conf": float(conf)
        })

# -------- Categorize detections --------
names = model.names  # class idx -> name
vehicle_labels = set(["car", "truck", "bus", "bike", "motorbike", "motorcycle"])  # extend as your model uses
plate_label_names = set(["license_plate", "plate", "number_plate"])
# car brand labels (as predicted by model) - include whatever classes your model has for logos
brand_labels = set([
    'nissan', 'toyota', 'isuzu', 'mitsubishi', 'proton', 'mazda', 'mercedes', 'honda', 'ford','bmw','audi','chevrolet'])

vehicles = []
plates = []
logos = []

for det in detections:
    box = det["box"]
    cls = det["class"]
    conf = det["conf"]
    label = names.get(cls, str(cls)).lower()
    if label in vehicle_labels:
        vehicles.append((label, box, conf, cls))
    elif label in plate_label_names:
        plates.append((label, box, conf, cls))
    elif label in brand_labels:
        logos.append((label, box, conf, cls))
    else:
        # try heuristic: if name contains 'plate' -> plate, contains brand substrings -> logo
        if "plate" in label:
            plates.append((label, box, conf, cls))
        elif any(b in label for b in ["nissan","toyota","honda","logo","mercedes","ford","bmw","audi","chev","mazda","isuzu","proton"]):
            logos.append((label, box, conf, cls))
        else:
            # ignore or keep for debug
            pass

# -------- Associate plates/logos with vehicles --------
scene_summary = {
    "file": os.path.basename(TEST_IMG),
    "timestamp": datetime.now().isoformat(),
    "incoming_traffic": False,
    "outgoing_traffic": False,
    "total_vehicles": 0,
    "vehicles": []
}

font = try_load_font(preferred_size=18)  # try to load TTF for nicer rendering

for vlabel, vboxf, vconf, vcls in vehicles:
    vbox = safe_int_box(vboxf)
    x1,y1,x2,y2 = vbox
    if x2 <= x1 or y2 <= y1:
        continue
    cx, cy = int((x1+x2)/2), int((y1+y2)/2)
    lane = "Left" if cx < w//2 else "Right"
    direction = "Incoming" if cy < h//2 else "Outgoing"

    # Vehicle crop and color
    vehicle_crop = orig_img[y1:y2, x1:x2].copy()
    color = mean_hsv_color_name(vehicle_crop)

    vehicle_info = {
        "type_class": vlabel,
        "detection_confidence": round(vconf, 4),
        "color": color,
        "make": "Unknown",
        "logo_bbox": None,
        "vehicle_bbox": [x1, y1, x2, y2],
        "license_plate_present": "No",
        "license_plate_bbox": None,
        "license_plate_color": None,
        "lane": lane,
        "direction": direction,
        "notes": []
    }

    # Match license plate: prefer plate center inside vehicle OR IOU > threshold
    matched_plate = None
    for plabel, pboxf, pconf, pcls in plates:
        pbox = safe_int_box(pboxf)
        if iou(vbox, pbox) > IOU_ASSOC_THRESH:
            matched_plate = (plabel, pbox, pconf)
            break
        # center-in-vehicle test
        pcx,pcy = box_center(pbox)
        if (pcx >= x1 and pcx <= x2 and pcy >= y1 and pcy <= y2):
            matched_plate = (plabel, pbox, pconf)
            break

    if matched_plate:
        plabel, pbox, pconf = matched_plate
        px1,py1,px2,py2 = pbox
        plate_crop = orig_img[py1:py2, px1:px2].copy()
        plate_color = mean_hsv_color_name(plate_crop)
        vehicle_info["license_plate_present"] = "Yes"
        vehicle_info["license_plate_bbox"] = [px1,py1,px2,py2]
        vehicle_info["license_plate_color"] = plate_color
        vehicle_info["notes"].append(f"plate_conf={round(pconf,3)}")

    # Match logo: prefer logo center inside vehicle OR IOU
    matched_logo = None
    for llabel, lboxf, lconf, lcls in logos:
        lbox = safe_int_box(lboxf)
        if iou(vbox, lbox) > IOU_ASSOC_THRESH:
            matched_logo = (llabel, lbox, lconf)
            break
        lcx,lcy = box_center(lbox)
        if (lcx >= x1 and lcx <= x2 and lcy >= y1 and lcy <= y2):
            matched_logo = (llabel, lbox, lconf)
            break

    if matched_logo:
        llabel, lbox, lconf = matched_logo
        lx1,ly1,lx2,ly2 = lbox
        vehicle_info["make"] = (llabel if llabel != "logo" else "Unknown")
        vehicle_info["logo_bbox"] = [lx1,ly1,lx2,ly2]
        vehicle_info["notes"].append(f"logo_conf={round(lconf,3)}")

    # append and draw
    scene_summary["vehicles"].append(vehicle_info)

    # draw bbox: color by direction (green incoming, red outgoing)
    draw_color = (0,200,0) if direction == "Incoming" else (0,0,200)
    cv2.rectangle(img, (x1,y1), (x2,y2), draw_color, thickness=3)

    # text for top-left: show type, color, make (short)
    short_make = vehicle_info["make"]
    text = f"{vlabel} | {color} | {short_make} | {lane} | {vconf:.2f}"
    # Use PIL draw for nicer font if available
    if font is not None:
        img = draw_text_pil(
            img, text, (x1, max(0, y1-30)),
            font=ImageFont.truetype(font.path, 18) if hasattr(font, "path") else font,
            text_color=(255,255,255), bg_color=(0,0,0)
    )       
    else:
        cv2.putText(img, text, (x1, max(15, y1-10)), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0,0,0), 4, cv2.LINE_AA)
        cv2.putText(img, text, (x1, max(15, y1-10)), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255,255,255), 1, cv2.LINE_AA)

    # draw plate bbox
    if vehicle_info["license_plate_present"] == "Yes":
        px1,py1,px2,py2 = vehicle_info["license_plate_bbox"]
        cv2.rectangle(img, (px1,py1), (px2,py2), (0,255,255), 2)
        plate_text = f"Plate: {vehicle_info['license_plate_present']}"
        if font is not None:
            img = draw_text_pil(img, plate_text, (px1, py2 + 4), font=font,
                            text_color=(0,0,0), bg_color=(255,255,0))
        else:
            cv2.putText(img, plate_text, (px1, py2 + 16), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 3)
            cv2.putText(img, plate_text, (px1, py2 + 16), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255,255,255), 1)


    # draw logo bbox
    if vehicle_info["logo_bbox"]:
        lx1,ly1,lx2,ly2 = vehicle_info["logo_bbox"]
        cv2.rectangle(img, (lx1,ly1), (lx2,ly2), (255,0,0), 2)
        logo_text = f"{vehicle_info['make']}"
        if font is not None:
            img = draw_text_pil(img, logo_text, (lx1, max(0, ly1-24)), font=font, text_color=(255,255,255), bg_color=(0,0,255))
        else:
            cv2.putText(img, logo_text, (lx1, max(12, ly1-6)), cv2.FONT_HERSHEY_DUPLEX, 0.6, (0,0,0), thickness=3)
            cv2.putText(img, logo_text, (lx1, max(12, ly1-6)), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255,255,255), thickness=1)

# -------- Scene level info --------
scene_summary["total_vehicles"] = len(scene_summary["vehicles"])
for v in scene_summary["vehicles"]:
    # deduce incoming/outgoing from per-vehicle direction
    if v["direction"] == "Incoming":
        scene_summary["incoming_traffic"] = True
    if v["direction"] == "Outgoing":
        scene_summary["outgoing_traffic"] = True

scene_summary["incoming_traffic"] = "Yes" if scene_summary["incoming_traffic"] else "No"
scene_summary["outgoing_traffic"] = "Yes" if scene_summary["outgoing_traffic"] else "No"

# banner
banner = f"Vehicles: {scene_summary['total_vehicles']} | Incoming: {scene_summary['incoming_traffic']} | Outgoing: {scene_summary['outgoing_traffic']}"
# draw banner background rectangle
cv2.rectangle(img, (0,0), (w, 36), (0,0,0), -1)
cv2.putText(img, banner, (12, 24), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)

# -------- Save outputs --------
annotated_path = os.path.join(SAVE_DIR, "annotated_result.jpg")
json_path = os.path.join(SAVE_DIR, "scene_summary.json")
cv2.imwrite(annotated_path, img)
with open(json_path, "w") as f:
    json.dump(scene_summary, f, indent=4)

print(f"✅ Annotated image saved at: {annotated_path}")
print(f"✅ JSON summary saved at: {json_path}")
print("Scene Summary:", json.dumps(scene_summary, indent=2))


Loading model: /content/Vehicle_Training/yolov8_vehicle_model4/weights/best.pt
Running inference...
✅ Annotated image saved at: /content/inference_results/annotated_result.jpg
✅ JSON summary saved at: /content/inference_results/scene_summary.json
Scene Summary: {
  "file": "20250915.172302.451.J001A1.FL.RE.PATRIOT1513.NaN.jpg",
  "timestamp": "2025-09-28T07:08:05.593388",
  "incoming_traffic": "Yes",
  "outgoing_traffic": "Yes",
  "total_vehicles": 2,
  "vehicles": [
    {
      "type_class": "car",
      "detection_confidence": 0.9976,
      "color": "Gray",
      "make": "toyota",
      "logo_bbox": [
        1108,
        932,
        1161,
        967
      ],
      "vehicle_bbox": [
        889,
        477,
        1471,
        1165
      ],
      "license_plate_present": "Yes",
      "license_plate_bbox": [
        1082,
        991,
        1194,
        1043
      ],
      "license_plate_color": "Blue",
      "lane": "Right",
      "direction": "Outgoing",
      "notes": [
  